In [ ]:
import csv
import json
import os
import zipfile

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
from community import community_louvain
from kaggle.api.kaggle_api_extended import KaggleApi
from sklearn.metrics import adjusted_mutual_info_score, adjusted_rand_score

import algorithm.kcomm.graph_kClusterAlgorithm_functions as QCD
import algorithm.kcomm.graphFileUtility_functions as GFU

In [ ]:
# Initialize and authenticate the Kaggle API
api = KaggleApi()
api.authenticate()

In [3]:
data_dir = "../data"
competition = 'cm4ai-community-detection-benchmark'
data_path = os.path.join(data_dir, competition)
os.makedirs(data_dir, exist_ok=True)


In [4]:
output_dir = "../output"
os.makedirs(output_dir, exist_ok=True)

In [5]:
# Download all files from a competition (e.g., Titanic)
competition = 'cm4ai-community-detection-benchmark'
api.competition_download_files(competition, path=data_dir, force=True)

In [6]:
# Extract all files from the zip to the specified directory
zip_file_path = data_path + ".zip"
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(data_path)

In [7]:
args_dict = {
    "benchmark_gen":"football",
    "output_dir" : output_dir
}

beta0 = 5
gamma0 = -250
threshold = 0.2
qsize = 64

In [ ]:
run_profile="defaults"

if args_dict["benchmark_gen"] == 'karate':

    print(f"Using benchmark graph generated by: nx-karate")  
    
    run_label = "zacharys-karate-club"
    input_graph = "zacharys-karate-club"
    
    G = nx.karate_club_graph() 

    gt_arr = []
    gt_arr = [G.nodes[v]['club'] for v in G.nodes()]
    gt_arr = [0 if x == 'Mr. Hi' else 1 for x in gt_arr]        # Convert to binary labels


elif args_dict["benchmark_gen"] == 'networkx':
    
    print(f"Using benchmark graph generated by: networkx")    

    run_label = "LFR_rs11_N1000_ad5_mc20_mu0.1"
    input_graph = f"../data/cm4ai-community-detection-benchmark/{run_label}"

    G = nx.read_edgelist(f"{input_graph}.edgelist")

    df = pd.read_csv(f"{input_graph}_communities.csv")
    gt_dict = df.set_index('id')['solution'].to_dict()

    sorted_by_keys = dict(sorted(gt_dict.items()))
    gt_arr = []
    for k,v in sorted_by_keys.items():
        gt_arr.append(v)

elif args_dict["benchmark_gen"] == 'dynbench':

    print(f"Using benchmark graph generated by: Dynbench")    
    
    # n = Number of nodes per community
    # q = Number of communities
    run_label = "stdmerge-n32-q8-pout01.t00100"
    input_graph = f"../data/cm4ai-community-detection-benchmark/{run_label}.graph"
    ground_truth_path = f"../data/scm4ai-community-detection-benchmark/{run_label}.comms"

    edgelist = pd.read_csv(input_graph, sep=' ', names=["source","target"])
    G = nx.from_pandas_edgelist(edgelist)
    for edge in G.edges():
        G[edge[0]][edge[1]]['weight'] = 1

    gt_arr=[]
    with open(ground_truth_path) as ground_truth_file:
        for line in ground_truth_file:
            if line.startswith("#"):
                continue
            fields = line.strip().split(" ")
            gt_arr.append(fields[1])
elif args_dict["benchmark_gen"] == 'football':

    print(f"Using benchmark graph generated by: Football")    
    
    run_label = "football"
    input_graph = f"../data/cm4ai-community-detection-benchmark/football_adjacency_matrix.csv"
    ground_truth_path = f"../data/cm4ai-community-detection-benchmark/football_labels.csv"
    adj_matrix = np.loadtxt(input_graph, delimiter=",", skiprows=1)
    print (adj_matrix.shape)
    # Create a graph from the adjacency matrix
    G = nx.from_numpy_matrix(adj_matrix)

    gt_arr=[]
    with open(ground_truth_path) as ground_truth_file:
        reader = csv.reader(ground_truth_file)
        next(reader)  # Skip the header
        for row in reader:
            gt_arr.append(int(row[1]))  # Add the second field (index 1) to the array


In [ ]:
nx.draw(
    G, 
    node_size=50,
    width=0.5,
    node_color="grey"
)

In [ ]:
A = nx.adjacency_matrix(G)
print ('\nAdjacency matrix:\n', A.todense())

num_parts = len(np.unique(gt_arr))
num_blocks = num_parts 
num_nodes = nx.number_of_nodes(G)
num_edges = nx.number_of_edges(G)
print (f"\nQuantum Community Detection: Up to {num_parts} communities")
print (f"Graph has {num_nodes} nodes and {num_edges} edges")

In [11]:
beta, gamma, GAMMA  = QCD.set_penalty_constant(num_nodes, num_blocks, beta0, gamma0)

In [ ]:
mtotal, modularity = QCD.build_mod(A, threshold, num_edges)
print ("\nModularity matrix: \n", modularity)

print ("min value = ", modularity.min())
print ("max value = ", modularity.max())

print ("threshold = ", threshold)

In [13]:
Q = QCD.makeQubo(G, modularity, beta, gamma, GAMMA, num_nodes, num_parts, num_blocks, threshold)

In [ ]:
result = {}
result['num_clusters'] = num_parts 
result['nodes'] = num_nodes
result['edges'] = num_edges
result['size'] = num_nodes * num_parts 
result['subqubo_size'] = qsize
result

In [ ]:
# Run k-clustering with Hybrid/D-Wave using ocean
ss = QCD.clusterHybrid(Q, num_parts, qsize, run_label, run_profile, result)
result

In [ ]:
# Process solution
part_number = QCD.process_solution(ss, G, num_blocks, num_nodes, num_parts, result)

In [ ]:
mmetric = QCD.calcModularityMetric(mtotal, modularity, part_number)
result['modularity_metric'] = mmetric

In [ ]:
# draw graph clusters and save .png
GFU.showClusters(part_number, G, args_dict)

In [ ]:
# write comms file 
GFU.write_partFile(
    part_num=part_number, 
    Dim=num_nodes, 
    nparts=num_parts, 
    args_dict=args_dict
) 

In [ ]:
columns = ["node_id", "comm_id"]
communities = []

pred_arr=[]

comm_file_path = os.path.join(args_dict['output_dir'], f"comm{num_parts}.txt")
with open(comm_file_path) as comm_file:
    i = 0
    for line in comm_file:
        i += 1
        if i == 1:
            continue
        fields = line.strip().split("  ")
        communities.append(fields)
        pred_arr.append(fields[1])

pred_arr = [int(x) for x in pred_arr]
pred_arr[:10]

In [ ]:
gt_arr[:10]

In [ ]:
# modularity = nx.community.modularity(G, pred_arr)
print(f"Modularity: {result['modularity_metric']}")
result['ari_score'] = adjusted_rand_score(gt_arr, pred_arr)
print(f"ARI: {result['ari_score']}")
result['ami_score'] = adjusted_mutual_info_score(gt_arr,pred_arr)
print(f"AMI: {result['ami_score']}")

In [ ]:
# --- Louvain Community Detection ---
# Perform Louvain community detection
louvain_partition = community_louvain.best_partition(G, resolution=1.5)
num_communities_louvain = len(set(louvain_partition.values()))
print("Number of communities (Louvain):", num_communities_louvain)

# --- Calculate Modularity ---
modularity_score = community_louvain.modularity(louvain_partition, G)
print(f"Modularity: {modularity_score}")

louvain_pred_arr = [louvain_partition[node] for node in G.nodes]

# --- Calculate ARI and AMI ---
ari_score = adjusted_rand_score(gt_arr, louvain_pred_arr)
ami_score = adjusted_mutual_info_score(gt_arr, louvain_pred_arr)
print(f"ARI: {ari_score}")
print(f"AMI: {ami_score}")

